In [27]:
import csv
import pprint
import pandas as pd
import pywikibot
import re
from string import punctuation
import urllib.request, json
import urllib.parse

from caphi2ipa import caphipp2ipa

# To do

* Singular nouns:
    * Add plurals
    * Handle "complex" nouns:
        * Several glosses (';')
        * Complex glosses ('_' or ' ')
    * Add IPA
* Plural nouns
* Adjectives
* Active participles (NOUN_ACT:MS)
* Passive participles (NOUN_PASS)
* INTERJ
* ADV
* NOUN:PHRASE
* Verbs
* VERB:PHRASE
* Issues:
    * شرعية: WARNING: API warning (edit): The value passed for "text" contains invalid or non-normalized data. Textual data should be valid, NFC-normalized Unicode without C0 control characters other than HT (\t), LF (\n), and CR (\r).

# Read TSV

In [4]:
maknuune_file = "maknuune-v1.0.1.tsv"

singular_nouns = ['NOUN:MS', 'NOUN:FS']

df = pd.read_csv(maknuune_file,sep='\t')

# Stats by PoS

In [5]:
df_count = df.groupby(['ANALYSIS'])['ANALYSIS'].count()
df_count.loc[lambda x : x > 50].sort_values(ascending=False)

ANALYSIS
VERB:I             6345
VERB:C             6329
VERB:P             6191
NOUN:MS            4183
NOUN:FS            3050
ADJ:MS             2388
NOUN:PHRASE        2343
NOUN:P             1894
NOUN_ACT:MS         759
VERB:PHRASE         639
NOUN_PASS           305
ADJ:P               288
ADJ:PHRASE          249
NOUN_ACT:PHRASE     117
ADJ/NOUN            108
INTERJ              106
ADJ:FS               78
ADJ_COMP             77
ADV                  71
NOUN_PROP            65
Name: ANALYSIS, dtype: int64

# First: singular nouns deriving from a root and without notes 

In [6]:
# 1355 entries with notes => deal with them later
# df.loc[df['NOTES'].notnull()].count()

# 897 entries with NTWS roots => deal with them later
# df.loc[df['ROOT'] == 'NTWS'].count()

# df['LEMMA_BW'] == df['FORM_BW'] to exclude weird cases for nouns

df_singular_nouns = df.loc[(df['ROOT'] != 'NTWS') & (df['ANALYSIS'].isin(singular_nouns)) & (df['LEMMA_BW'] == df['FORM_BW']) & (df['NOTES'].isnull())]

df_singular_nouns

,ID,ROOT,ROOT_NTWS,ROOT_1,LEMMA,LEMMA_SEARCH,FORM,LEMMA_BW,FORM_BW,CAPHI++,ANALYSIS,GLOSS,GLOSS_MSA,EXAMPLE_USAGE,NOTES,SOURCE,ANNOTATOR,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,1,ء.ب.د,NaN,ء,أَبَد,أبد,أَبَد,>abad,>abad,2 a b a d,NOUN:MS,eternity;forever,NaN,NaN,NaN,NaN,شهد دعباس,NaN,NaN,NaN
1,2,ء.ب.ر,NaN,ء,إِبْرِة,إبرة,إِبْرِة,<iborip,<iborip,2 i b r e,NOUN:FS,needle;injection,إِبْرَة#حُقْنَة,في إِبْرِة وقعت تحت الكنب دير بالك.#أخذت ابرة ...,NaN,NaN,شهد دعباس,NaN,NaN,NaN
7,8,ء.ب.ط,NaN,ء,أَبَاط,أباط,أَبَاط,>abaAT,>abaAT,2 a b aa t.,NOUN:MS,armpit,إِبْط,ولك ليش أباطك أسود هيك؟,NaN,NaN,شهد دعباس,NaN,NaN,NaN
8,9,ء.ب.ط,NaN,ء,بَاط,باط,بَاط,baAT,baAT,b aa t.,NOUN:MS,armpit,إِبْط,NaN,NaN,NaN,شهد دعباس,NaN,NaN,NaN
10,11,ء.ب.و,NaN,ء,أَب,أب,أَب,>ab,>ab,2 a b,NOUN:MS,father,أب,الله يرحمه أبوه بقى زلمة مليح وبينشد فيه الظهر,NaN,NaN,شهد دعباس,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36039,36263,ي.م.ن,NaN,ي,يَمِين,يمين,يَمِين,yamiyn,yamiyn,y a m ii n,NOUN:MS,right,يمين,روتش لك شوي لليمين,NaN,NaN,شهد دعباس,NaN,NaN,NaN
36040,36264,ي.م.ن,NaN,ي,يَمِين,يمين,يَمِين,yamiyn,yamiyn,y a m ii n,NOUN:MS,vow;oath,NaN,احلف يَمِين إِنه مالكاش دخَّل!,NaN,NaN,شهد دعباس,NaN,NaN,NaN
36044,36268,ي.م.ن,NaN,ي,يُمُن,يمن,يُمُن,yumun,yumun,y u m u n,NOUN:MS,blessing,نِعْمَة,ألف مبارك الشهر وكل عام وانتو بخير وان شاء الل...,NaN,NaN,شهد دعباس,NaN,NaN,NaN
36056,36280,ي.و.م,NaN,ي,يَومِيِّة,يومية,يَومِيِّة,yawmiy~ip,yawmiy~ip,y oo m i y y e,NOUN:FS,daily_payment,دفع بنظام المياومة,NaN,NaN,NaN,شهد دعباس,NaN,NaN,NaN


# Wiktionary format

In [9]:
beginning_file = """'''User:A455bcd9/Maknuune/{entry}'''
{{{{-start-}}}}"""

template = """==South Levantine Arabic==
{{{{bulk import|{{{{R:ajp:Maknuune|id={id}}}}}}}}}
{{{{ajp-root|{root}}}}}

===Pronunciation===

* {{{{ajp-IPA|{ipa}}}}}

===Noun===
{{{{ajp-noun|head={head}|g={gender}|tr={transliteration}{plural}}}}}

# {glosses}{example}

===References===
* {{{{R:ajp:Maknuune|id={id}}}}}"""

end_file="{{{{-stop-}}}}"

"""
Plural:

===Noun===
{{ajp-noun|head={head}|g=p|tr={transliteration}}}
# {{plural of|ajp|{singular}|tr={singulartransliteration}|t={singulargloss}}}
# [[definition]]

"""

'\nPlural:\n\n===Noun===\n{{ajp-noun|head={head}|g=p|tr={transliteration}}}\n# {{plural of|ajp|{singular}|tr={singulartransliteration}|t={singulargloss}}}\n# [[definition]]\n\n'

# Select 10 random entries

In [99]:
# df_singular_nouns = df_singular_nouns.sample(n=1000)

# CAPHI++ to Wikt

In [10]:
for index, row in df_singular_nouns.iterrows():
    caphipp = row['CAPHI++'].strip()
    ipa = caphipp2ipa(caphipp, mode='wikt')
    # print(row['LEMMA'], '=>', caphipp, '=>', ipa)
    
# test m a w J uu d => mawjūd (موجود)
# maw(dʒ)uːd // مَوْ ُجود
print(caphipp2ipa('m a w J uu d', mode='wikt'))
# 2 i sh t aa Q => ištāʔ (اشتاق)
# ʔiʃtaː(q)
print(caphipp2ipa('2 i sh t aa Q', mode='wikt'))

mawjūd
ʔištāʔ


# Test entries

In [11]:
test_entries = [
    'أب',
    'أباط',
    'أجار',
    'أجرة',
    'أخذ',
    'ألفة',
    'إيجار',
    'باط',
    'بلع',
    'تأثير',
    'تأجير',
    'تأجيل',
    'تجارة',
    'خرقة',
    'زهاب',
    'عنصرية',
    'غواصة',
    'قذلة',
    'كماجة',
    'لقمة',
    'مهاجر',
    'مواخذة',
    'شرعية',
    'بيئة',
    'كيلو',
    'مفاد',
    'خضرة',
    'نظرة',
    'فناء',
    'إفشاء',
    'عجلة',
    'صحصحة',
    'واعظ',
]

# Save entries in Wiktionary

## Clean glosses

In [79]:
def clean_gloss(gloss):
    # Clean and link glosses
    clean_gloss = gloss
    
    replacements = [
        (u"\u2018", "'"),
        (u"\u2019", "'"),
        ("_", " "),
        (r' +', ' ',),
        ('sb/sth', 'somebody or something'),
        ("(?i) sth", " something"),
        ("(?i) sb", " somebody"),
        (r"(?i)^sb ", "somebody "),
        (r"(?i)^sth ", "something "),
        ("(?i) some one ", " someone "),
        (r"(?i)^a type of ", ""),
        (r"(?i)^see phrase(s)?", ""),
        (r"(?i)^a ", ""),
        (r"(?i)^an ", ""),
        (r"(?i)^The ", ""),
        (r"(?i)^It is what ", "what "),
        (r"(?i)^It is (a|the) ", ""),
        (r"(?i)^It is like (a|the) ", ""),
        (' that is made of ', ' made of '),
        (' that is made from ', ' made from '),
        (' a n ', ' an ')
    ]
    
    for replacement in replacements:
        clean_gloss = re.sub(replacement[0], replacement[1], clean_gloss)
        
    clean_gloss = clean_gloss.strip(' .')
    
    return clean_gloss

def link_gloss(gloss):
    # clean gloss
    linked_gloss = clean_gloss(gloss)
    
    # if single word, link it
    if len(linked_gloss.split(' ')) == 1 and linked_gloss != '':
        linked_gloss = u"[[{g}]]".format(g=linked_gloss)
    elif len(linked_gloss.split(' ')) in [2, 3]:
        # Check whether English n-grams (glosses) are already in the Wiktionary and link them
        api_url = u"https://en.wiktionary.org/w/api.php?action=query&titles={entry}&format=json".format(entry=linked_gloss).replace(' ', '%20')
        with urllib.request.urlopen(api_url) as url:
            data = json.load(url)
            # if the entry exists, link it
            if not data.get('query').get('pages').get('-1'):
                linked_gloss = u"[[{g}]]".format(g=linked_gloss)
    
    return linked_gloss

In [61]:
link_gloss('narrow path')
link_gloss('take down')

'[[take down]]'

In [17]:
def test_clean_gloss():
    to_test = [
        ('a_type_of_jar_for_keeping_water_cold', 'jar for keeping water cold'),
        ('see_phrase', ''),
        ('ceramic work.', 'ceramic work'),
        ('sb_who_eats_food_off_the_ground', 'somebody who eats food off the ground'),
        ('a small ball', 'small ball'),
        ('a single person', 'single person'),
        ('a narrow path', 'narrow path'),
        ('a fat person', 'fat person'),
        ('sb/sth', 'somebody or something'),
        ('a difficult  situation', 'difficult situation'),
        ('the grated food', 'grated food'),
        ('the beautiful woman', 'beautiful woman'),
        ('a_stone_inside_a_hole', 'stone inside a hole'),
        ('an individual.', 'individual'),
        ('see phrases', ''),
        ("the_state_of_holding_something_over_sb's_head", "state of holding something over somebody's head"),
        ('a_vessel_that_is_usually_of_brass_for_kohl.', 'vessel that is usually of brass for kohl'),
        ('It_is_what_is_placed_on_the_back_of_the_walking_animal_for_the_purpose_of_riding_on_it', 'what is placed on the back of the walking animal for the purpose of riding on it'),
        ('It_is_the_ritual_animal_sacrifice_of_a_livestock_animal_in_the_first_three_days_of_the_funeral_that_is_believed_to_give_solace_to_the_deceased_person', 'ritual animal sacrifice of a livestock animal in the first three days of the funeral that is believed to give solace to the deceased person'),
        ('A_thin_layer_of_freezing_water_in_the_morning,_due_to_lower_temperatures.', 'thin layer of freezing water in the morning, due to lower temperatures'),
        ('It_is_a_traditional_type_of_dessert_that_is_made_of_dough,_water_and_molasses._It_is_usually_eaten_with_dried_figs.', 'traditional type of dessert made of dough, water and molasses. It is usually eaten with dried figs'),
        ('sth_similar_to_a_container_that_is_made_by_sewing_several_"7_u_s._u_r"_together._The_farmers_usually_keep_wheat_in_it.', 'something similar to a container that is made by sewing several "7 u s. u r" together. The farmers usually keep wheat in it'),
        ('the_person_whose_job_is_to_tell_stories_to_the_people', 'person whose job is to tell stories to the people'),
        ('It_is_like_a_basket_made_of_rubber,_or_of_wicker_and_fiber,_and has two loops that are used as handles for carrying it. People use it to carry multiple things.', 'basket made of rubber, or of wicker and fiber, and has two loops that are used as handles for carrying it. People use it to carry multiple things'),
        ('traditional Levantine dish that is made of chickpeas', 'traditional Levantine dish made of chickpeas'),
        ('small container that is made from skinned sheepskin', 'small container made from skinned sheepskin'),
        ('speak_in_a_n_incomprehensible_way', 'speak in an incomprehensible way'),
    ]
    for unit_test in to_test:
        if clean_gloss(unit_test[0]) != unit_test[1]:
            raise ValueError('Unit test failed:', clean_gloss(unit_test[0]), unit_test[1])
            
    return 'clean_gloss(): OK'

test_clean_gloss()

'clean_gloss(): OK'

In [80]:
# site = pywikibot.Site()
i = 0

for index, row in df_singular_nouns.iterrows():
    # remove _[auto] from entries generated automatically from other entries
    clean_gloss_row = row['GLOSS'].replace('_[auto]', '').strip()
    # filter out more complex glosses
    entry_values = {}
    entry_values['id'] = row['ID']

    entry_values['entry'] = row['LEMMA_SEARCH'].strip()
    entry_values['root'] = row['ROOT'].strip().replace('.', ' ')
    entry_values['head'] = row['FORM'].strip()
    entry_values['gender'] = 'f' if row['ANALYSIS'] == 'NOUN:FS' else 'm'

    caphipp = row['CAPHI++'].strip()
    entry_values['transliteration'] = caphipp2ipa(caphipp, mode='wikt')
    entry_values['ipa'] = entry_values['transliteration']
    
    entry_values['plural'] = ''

    entry_values['example'] = ''
    if not pd.isna(row['EXAMPLE_USAGE']):
        example = row['EXAMPLE_USAGE'].strip()
        # Fill example if not null
        if example != '':
            entry_values['example'] = '\n#: {{{{ux|ajp|{example}}}}}'.format(example=example)

    # Separate glosses
    glosses = set(clean_gloss_row.split(';'))
    
    # Do not add empty glosses
    if len(glosses) == 1:
        if link_gloss(list(glosses)[0]) == '':
            continue
        
    # clean glosses and link simple ones
    entry_values['glosses'] = ', '.join([link_gloss(x) for x in glosses])

    new_entry = template.format(**entry_values)

    to_save = False
    if to_save:
        page_name = u"User:A455bcd9/Maknuune/{entry}".format(entry=entry_values['entry'])
        page = pywikibot.Page(site, page_name)

        page.text = new_entry
        print(new_entry)
        page.save(u"Entries with multiple glosses and examples")
        i += 1    
            
    if i > 20:
        break

print('end')

eternity;forever
1 [[forever]], [[eternity]]
needle;injection
2 [[injection]], [[needle]]
armpit
8 [[armpit]]
armpit
9 [[armpit]]
father
11 [[father]]
daddy;father_Pope
28 [[daddy]], father Pope
daddy;father
29 [[daddy]], [[father]]
the_father_of;the_possessor_of;having_a_particular_quality
30 having a particular quality, possessor of, father of
furnishing_(a_place)
36 furnishing (a place)
trace;effect
43 [[trace]], [[effect]]
effect
44 [[effect]]
fare
61 [[fare]]
rent
63 [[rent]]
rent
64 [[rent]]
rent
65 [[rent]]
Hasanat_(Credit_for_good_deeds,_which_Allah_weighs_up_against_one's_bad_deeds_at_the_final_judgement_after_death)
66 Hasanat (Credit for good deeds, which Allah weighs up against one's bad deeds at the final judgement after death)
period;term
77 [[term]], [[period]]
postponement
78 [[postponement]]
taking
87 [[taking]]
blame;shortcoming
88 [[shortcoming]], [[blame]]
taking_sth
107 taking something
blaming
108 [[blaming]]
blaming
109 [[blaming]]
The_Day_of_the_Judgment
119 Day

1076 saying or so something randomly, saying or so something on an adhoc basis, waffling on something
bargain;haggle
1080 [[haggle]], [[bargain]]
witch
1090 [[witch]]
tower
1091 [[tower]]
wearing_make-up
1092 wearing make-up
the_person_who_walks_from_place_to_place_(sometimes_he_rides_a_donkey)_selling_fabrics
1093 person who walks from place to place (sometimes he rides a donkey) selling fabrics
coo;speaking_in_a_n_incomprehensible_way
1104 speaking in an incomprehensible way, [[coo]]
yard;courtyard
1111 [[courtyard]], [[yard]]
curtain
1122 [[curtain]]
gun
1123 [[gun]]
the_teapot
1125 [[teapot]]
the_state_of_being_stale
1126 state of being stale
cold;coldness
1127 [[cold]], [[coldness]]
It_is_what_is_placed_on_the_back_of_the_walking_animal_for_the_purpose_of_riding_on_it._It_is_somewhat_similar_to_a_horse's_saddle;donkey-saddle
1149 [[donkey-saddle]], what is placed on the back of the walking animal for the purpose of riding on it. It is somewhat similar to a horse's saddle
land;eart

1733 [[small ball]]
gaining_weight;becoming_ball-like_(especially_dough)
1735 becoming ball-like (especially dough), [[gaining weight]]
sending_sth
1743 sending something
scholarship
1744 [[scholarship]]
it_is_a_dish_that_is_made_of_fried_potatoes_with_eggs,_salt_and_some_black_pepper.
1752 dish made of fried potatoes with eggs, salt and some black pepper
scattering_things
1753 scattering things
separate;infrequent
1774 [[infrequent]], [[separate]]
deportation;exile
1776 [[deportation]], [[exile]]
picking_olives_after_the_main_harvest
1807 picking olives after the main harvest
camel
1808 [[camel]]
excrement;shit
1809 [[excrement]], [[shit]]
the_drum_that_the_"m_s_a_7_7_a_r_aa_t_i"_uses_in_order_to_wake_people_up_for_"s_u_7_uu_r"_("m_s_a_7_7_a_r_aa_t_i"the_man_whose_job_is_to_wake_the_people_up_to_have_their_"s_u_7_uu_r"_in_Ramadan_(i.e._the_last_meal_taken_before_daybreak_during_Ramadan)
1818 drum that the "m s a 7 7 a r aa t i" uses in order to wake people up for "s u 7 uu r" ("m s a 

2938 [[sadness]], sad event
tailor
2947 [[tailor]]
terrace
2953 [[terrace]]
terrace
2954 [[terrace]]
the_state_of_being_filled_to_the_max_or_stuffed
2955 state of being filled to the max or stuffed
speaking_fluently
2965 speaking fluently
a_passage
2969 [[passage]]
bequeath;heritage
2975 [[bequeath]], [[heritage]]
earing
2976 [[earing]]
leaving;abandoning
2978 [[leaving]], [[abandoning]]
tiredness
3008 [[tiredness]]
suffering
3020 [[suffering]]
stuttering;stammering
3024 [[stammering]], [[stuttering]]
unhappiness
3032 [[unhappiness]]
spitting_a_lot;salivating
3052 [[salivating]], spitting a lot
spit
3065 [[spit]]
spit
3073 [[spit]]
salivating
3074 [[salivating]]
triviality;trifle
3083 [[trifle]], [[triviality]]
toktok
3095 [[toktok]]
stubbornness
3101 [[stubbornness]]
second;minute
3108 [[second]], [[minute]]
free_eating-place;philanthropic_institution
3109 free eating-place, philanthropic institution
ashtray
3110 [[ashtray]]
the_state_of_being_thick-skinned;insesitive
3122 state of be

4260 loose gown
cattle_feed
4262 cattle feed
robe
4263 [[robe]]
resounding_noise
4292 resounding noise
sb_or_sth_that_sharpens_an_item_made_of_iron;sharpener
4298 [[sharpener]], somebody or something that sharpens an item made of iron
sharpening_sth
4302 sharpening something
clear_book_cover
4312 clear book cover
stamina
4313 [[stamina]]
flogging;whipping
4314 [[flogging]], [[whipping]]
the_number_of_times_sb_is_being_flogged/whipped
4315 number of times somebody is being flogged/whipped
the_outer_skin
4316 outer skin
the_outer_skin;leather
4317 [[leather]], outer skin
council
4336 [[council]]
sitting;session;hearing
4337 [[session]], [[sitting]], [[hearing]]
stroke
4352 [[stroke]]
the_state_of_making_food_overcooked_and_mushy
4359 state of making food overcooked and mushy
quaffing;chugging;drinking_water_quickly
4370 [[chugging]], drinking water quickly, [[quaffing]]
unfriendliness;unsmiling_face
4379 unsmiling face, [[unfriendliness]]
grimacing_at_sb
4385 grimacing at somebody
mouth


...

32409 deceiving some one, putting up something, pitching something
cupboard_where_mattresses_and_blankets_are_folded_and_kept
32412 cupboard where mattresses and blankets are folded and kept
stove_stand
32413 stove stand
deceiver
32414 [[deceiver]]
destiny;share
32415 [[share]], [[destiny]]
erection
32416 [[erection]]
fatness;obesity
32434 [[fatness]], [[obesity]]
advice
32435 [[advice]]
Christianity
32459 [[Christianity]]
truimph;victory
32460 [[victory]], [[truimph]]
truimph;victory
32461 [[victory]], [[truimph]]
big_tin_can_for_washing_clothes
32480 big tin can for washing clothes
big_tin_can_used_for_preserving_(food)
32481 big tin can used for preserving (food)
text
32482 [[text]]
half
32500 [[half]]
fairness;equity;justice
32501 [[justice]], [[equity]], [[fairness]]
maturity;ripeness;welllcooked
32518 [[maturity]], [[welllcooked]], [[ripeness]]
it_is_a_small_place_like_a_pergola_that_is_built_from_stones_and_covered_with_leaves._Usually,_the_guards_stay_in_it_in_order_to_watch_th

33353 nibbling at something, eating small bites or small quantites of food from all the dishes, getting a bite to eat
getting_a_bite_to_eat;eating_small_bites_or_small_quantites_of_food_from_all_the_dishes;nibbling_at_sth
33354 nibbling at something, eating small bites or small quantites of food from all the dishes, getting a bite to eat
selection
33359 [[selection]]
purity
33360 [[purity]]
affliction
33374 [[affliction]]
joke
33381 [[joke]]
stinginess
33382 [[stinginess]]
wedlock;copulation
33399 [[wedlock]], [[copulation]]
the_state_of_being_peevish;moody;ill-tempered
33405 state of being peevish, [[ill-tempered]], [[moody]]
nothing;nobody
33418 [[nobody]], [[nothing]]
denial
33419 [[denial]]
masquerade
33420 [[masquerade]]
repudiation;disapprobation
33433 [[repudiation]], [[disapprobation]]
tingle;prick
33439 [[tingle]], [[prick]]
setback;embarrassing_situation
33445 embarrassing situation, [[setback]]
relapse
33447 [[relapse]]
twig;swab
33464 [[twig]], [[swab]]
raising_an_issue;jok

34331 helve or haft, hang of an ax
mock
34341 [[mock]]
shake
34361 [[shake]]
shaking;dancing
34362 [[dancing]], [[shaking]]
comedy;farce
34371 [[farce]], [[comedy]]
defeat
34379 [[defeat]]
jiggle;shake
34380 [[jiggle]], [[shake]]
silence
34389 [[silence]]
craziness
34396 [[craziness]]
a_wild_plant_that_has_sharp_points
34409 wild plant that has sharp points
handheld_folding_fan;handheld_piece_of_cardboard
34411 handheld folding fan, handheld piece of cardboard
swatting;driving_the_flies_away;waving_the_handheld_folding_fan_or_handheld_piece_of_cardboard
34412 driving the flies away, [[swatting]], waving the handheld folding fan or handheld piece of cardboard
the_state_of_packing_one's_perosanl_belongings_and_leaving
34418 state of packing one's perosanl belongings and leaving
smashing_sth;crush_sth
34423 smashing something, crush something
waddling_down
34430 waddling down
digestion
34437 [[digestion]]
shower;rainfall
34459 [[shower]], [[rainfall]]
rushing
34487 [[rushing]]
blowing_pas

35243 [[packing away]], [[gorging]], putting away
legacy;inheritance
35247 [[legacy]], [[inheritance]]
genetic
35248 [[genetic]]
legacy;inheritance
35249 [[legacy]], [[inheritance]]
legacy;inheritance;heritage
35251 [[legacy]], [[heritage]], [[inheritance]]
heir;inheritor
35252 [[heir]], [[inheritor]]
import
35270 [[import]]
morning_and_evening_Adhkar_that_are_recited_to_protect_the_person
35272 morning and evening Adhkar that are recited to protect the person
A_peel_is_a_shovel-like_tool_used_by_bakers_to_slide_loaves_of_bread,_pizzas,_pastries,_and_other_baked_goods_into_and_out_of_an_oven.
35288 peel is a shovel-like tool used by bakers to slide loaves of bread, pizzas, pastries, and other baked goods into and out of an oven
workshop
35294 [[workshop]]
entanglement
35306 [[entanglement]]
paper
35316 [[paper]]
paper
35317 [[paper]]
buttocks;chicken_thigh
35324 chicken thigh, [[buttocks]]
tumour
35330 [[tumour]]
behind
35353 [[behind]]
behind
35357 [[behind]]
minister
35366 [[minister

36280 daily payment
day
36281 [[day]]
end


In [42]:
"""
To check: 

New line:
2955 state of being filled to the max or stuffed
6194 traditional dish that is made of unripe grapes and lentils
35498 being handsome
"""

'\nissues:\n\nsee_phrases\nsee_phrase:\n33175\n33177 \n33179 \n735\n968\n1212\n2202\n13973\n\nNew line:\n2955 state of being filled to the max or stuffed\n6194 traditional dish that is made of unripe grapes and lentils\n35498 being handsome\n'